In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(123)
torch.manual_seed(123)

### Step 1: Generate Data

In [3]:
from synthetic.generate import SingleTaskTreeDepsGenerator

K = 3
M = 10
N = 10000

# Generate the true class balance to be recovered
class_balance = np.ones(K)/K + np.random.random(K)/5.
class_balance /= class_balance.sum()

#, edges=[(0,1)]
data = SingleTaskTreeDepsGenerator(N, M, K, class_balance, edges=[(0,1)])
print (f"LF Dependencies: {data.E}")
print (f"Class Balance: {data.p}")

LF Dependencies: [(0, 1)]
Class Balance: [0.38057027 0.31448886 0.30494087]


In [4]:
from metal.analysis import lf_summary
lf_summary(data.L,data.Y)

,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
0,"[1, 2, 3]",0.9067,0.9067,0.8697,7144,1923,0.787912
1,"[1, 2, 3]",0.8971,0.8971,0.8593,6991,1980,0.779289
2,"[1, 2, 3]",0.8256,0.8256,0.7954,4728,3528,0.572674
3,"[1, 2, 3]",0.8165,0.8165,0.7856,4404,3761,0.539375
4,"[1, 2, 3]",0.9187,0.9187,0.8795,7499,1688,0.816262
5,"[1, 2, 3]",0.8610,0.8610,0.8276,5879,2731,0.682811
6,"[1, 2, 3]",0.8260,0.8260,0.7975,4820,3440,0.583535
7,"[1, 2, 3]",0.8526,0.8526,0.8191,5619,2907,0.659043
8,"[1, 2, 3]",0.9076,0.9076,0.8702,7214,1862,0.794844
9,"[1, 2, 3]",0.8300,0.8300,0.7970,5033,3267,0.606386


### Step 2: Learn Dependencies using `DependencyLearnerModel`
**TEMP: hardcoding for now**

In [5]:
edges = data.E
L = np.array(data.L.todense())

### Step 3: Recover the class balance using subset of independent LFs

In [6]:
# Generate indices for independent LFs
ind_lfs = []
for i in range(M):
    if i not in list(sum(edges, ())):
        ind_lfs.append(i)
L_ind = L[:,ind_lfs]

In [7]:
%%time
from metal.label_model.class_balance import ClassBalanceModel

cb_model = ClassBalanceModel(K, abstains=True)
cb_model.train_model(L=L_ind, verbose=False)

print(f"Estimated class balance: {cb_model.class_balance}")
print(f"True class balance: {class_balance}")
print()

Estimated class balance: [0.38704416 0.319095   0.2936667 ]
True class balance: [0.38057027 0.31448886 0.30494087]

CPU times: user 10.6 s, sys: 72 ms, total: 10.7 s
Wall time: 2.51 s


### Step 4: Train `LabelModel` using $\mu$ from `ClassBalanceModel`

In [30]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(verbose=True,k=K, seed=123)

label_model.train_model(
    data.L, 
    cond_probs = cb_model.cond_probs, 
    ind_lfs = ind_lfs, 
    deps = edges, 
    #class_balance = cb_model.class_balance.astype(float),
    lr = 1e-3, 
    n_epochs = 1000, 
    log_train_every = 500,
    l2=5e-1
)

print()
for y in range(K):
    mu_idx = [il*K + y for il in ind_lfs]
    diff = np.linalg.norm(label_model.mu[mu_idx,y].detach().numpy() - 
                         label_model.mu_init[mu_idx,y].detach().numpy())/(M*1.0)
    print(f"Diff between mu and mu_init for y = {y+1}: {diff}")
print()

Computing O^{-1}...
Estimating Z...
[500 epo]: TRAIN:[loss=72.772]
[1000 epo]: TRAIN:[loss=71.906]
Finished Training
Estimating \mu...
[500 epo]: TRAIN:[loss=0.573]
[1000 epo]: TRAIN:[loss=0.573]
Finished Training

Diff between mu and mu_init for y = 1: 0.006093572080135346
Diff between mu and mu_init for y = 2: 0.014665958285331727
Diff between mu and mu_init for y = 3: 0.006208213046193123

CPU times: user 30.4 s, sys: 2.92 s, total: 33.3 s
Wall time: 2.02 s


In [31]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.951
Precision: 0.942
Recall: 0.962
F1: 0.952
        y=1    y=2    y=3   
 l=1   3686    166    59    
 l=2    52    2947    29    
 l=3    93     88    2880   


### Step 4b: Compare to MajorityVote and No Class Balance+Dependencies LabelModel

**Baseline: No Class Balance or Dependencies**

In [10]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, lr=1e-3, n_epochs=500, log_train_every=50)

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.150]
[100 epo]: TRAIN:[loss=0.058]
[150 epo]: TRAIN:[loss=0.029]
[200 epo]: TRAIN:[loss=0.017]
[250 epo]: TRAIN:[loss=0.013]
[300 epo]: TRAIN:[loss=0.011]
[350 epo]: TRAIN:[loss=0.010]
[400 epo]: TRAIN:[loss=0.009]
[450 epo]: TRAIN:[loss=0.009]
[500 epo]: TRAIN:[loss=0.009]
Finished Training
CPU times: user 11.8 s, sys: 92 ms, total: 11.9 s
Wall time: 432 ms


In [11]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.955
Precision: 0.956
Recall: 0.957
F1: 0.957
        y=1    y=2    y=3   
 l=1   3668    131    36    
 l=2    68    2978    25    
 l=3    95     92    2907   


**Baseline: Majority Vote**

In [12]:
from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(k=K,seed=123)
print('Majority Label Voter Metrics:')
scores = mv.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Majority Label Voter Metrics:
Accuracy: 0.944
Precision: 0.951
Recall: 0.950
F1: 0.951
        y=1    y=2    y=3   
 l=1   3640    134    52    
 l=2    90    2937    51    
 l=3    101    130   2865   
